Training and forecasting the solar power of Hokkaido Island in Japna
Training 2017, Forecast 2018

https://stackoverflow.com/questions/35139108/how-to-install-xgboost-in-anaconda-python-windows-platform

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import csv 
import matplotlib.pyplot as plt
import pytz
import xgboost as xgb

In [2]:
import os
cwd = os.getcwd()
cwd
# os.chdir('pvinhokkaidodatadistribution')
# C:\Users\Mhdella\Google Drive\Jupyter Folder\PV_Hokkaido_forecast_contest\
os.chdir('C:\\Users\\Mhdella\\Google Drive\\Jupyter Folder\\PV_Hokkaido_forecast_contest\\pvinhokkaidodatadistribution')
nwd=os.getcwd()
nwd

'C:\\Users\\Mhdella\\Google Drive\\Jupyter Folder\\PV_Hokkaido_forecast_contest\\pvinhokkaidodatadistribution'

In [3]:
# file_name='Merg_Ens_Agg_42_44_mes17_18.csv'
# file_name='Merg_Ens_Agg_43_mes17_18.csv'

# file_name='Merg_Ens_Agg_42_44_mes16_17_18.csv'
file_name='Merg_Ens_mmaxmin_Agg_42_44_mes16_18.csv'

# file_name='Merg_Res_Comb_S1_S2_2018_tz_Jp.csv'
# file_mane='Merg_Res_Comb_S1_S2_2017_tz_Jp.csv'

df_hok=pd.read_csv(file_name) 
df_hok.head()
# df_hok.tail()

,DATE,TIME,S1[MW],S2[MW],solar[kW/m2],temp[deg C],solar[kW/m2].1,temp[deg C].1,solar[kW/m2].2,temp[deg C].2,...,t2m.2,asn.2,sd.2,ssrdc.2,ssrd.2,tcc.2,tciw.2,tclw.2,tp.2,wind_speed.2
0,1/1/2016,9:00:00,54.7,78.4,0.159228,-1.2,0.276627,0.0,0.339371,-5.9,...,-13.909399,0.600965,7.450000e-09,57.319398,30.413241,0.000000,0.000000,0.000000,0.000000e+00,7.704484
1,1/1/2016,9:30:00,113.8,89.0,0.221269,-0.3,0.307764,0.1,0.297246,-4.4,...,-12.694835,0.600870,7.450000e-09,104.414120,55.211142,0.037510,0.000396,0.000012,2.690000e-07,7.772519
2,1/1/2016,10:00:00,115.9,97.3,0.231619,-0.5,0.354492,0.7,0.320564,-2.3,...,-11.480271,0.600775,7.450000e-09,151.508843,80.009043,0.075021,0.000793,0.000023,5.390000e-07,7.840554
3,1/1/2016,10:30:00,139.5,100.6,0.205909,-0.4,0.363168,0.8,0.306980,-1.9,...,-10.265707,0.600680,7.450000e-09,198.603565,104.806944,0.112531,0.001189,0.000034,8.080000e-07,7.908589
4,1/1/2016,11:00:00,125.7,96.8,0.277444,-0.2,0.358989,0.6,0.237579,-3.1,...,-9.051143,0.600586,7.450000e-09,245.698287,129.604846,0.150041,0.001585,0.000046,1.080000e-06,7.976625


In [4]:
#### This cell just to check out the dataset with different coordination lat 43 and lat42-44 from ECMWF
df_hok1=pd.read_csv('Merg_Ens_Agg_42_44_mes17_18.csv') 
df_hok2=pd.read_csv('Merg_Ens_Agg_43_mes17_18.csv')

df_mns= {'col1': df_hok1.mean(), 'col2': df_hok2.mean()}
# df_mns

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-regression-py

In [5]:
import sklearn 
import numpy as np
from sklearn.metrics import mean_squared_error
# X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
# X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
df_hok.head()

,DATE,TIME,S1[MW],S2[MW],solar[kW/m2],temp[deg C],solar[kW/m2].1,temp[deg C].1,solar[kW/m2].2,temp[deg C].2,...,t2m.2,asn.2,sd.2,ssrdc.2,ssrd.2,tcc.2,tciw.2,tclw.2,tp.2,wind_speed.2
0,1/1/2016,9:00:00,54.7,78.4,0.159228,-1.2,0.276627,0.0,0.339371,-5.9,...,-13.909399,0.600965,7.450000e-09,57.319398,30.413241,0.000000,0.000000,0.000000,0.000000e+00,7.704484
1,1/1/2016,9:30:00,113.8,89.0,0.221269,-0.3,0.307764,0.1,0.297246,-4.4,...,-12.694835,0.600870,7.450000e-09,104.414120,55.211142,0.037510,0.000396,0.000012,2.690000e-07,7.772519
2,1/1/2016,10:00:00,115.9,97.3,0.231619,-0.5,0.354492,0.7,0.320564,-2.3,...,-11.480271,0.600775,7.450000e-09,151.508843,80.009043,0.075021,0.000793,0.000023,5.390000e-07,7.840554
3,1/1/2016,10:30:00,139.5,100.6,0.205909,-0.4,0.363168,0.8,0.306980,-1.9,...,-10.265707,0.600680,7.450000e-09,198.603565,104.806944,0.112531,0.001189,0.000034,8.080000e-07,7.908589
4,1/1/2016,11:00:00,125.7,96.8,0.277444,-0.2,0.358989,0.6,0.237579,-3.1,...,-9.051143,0.600586,7.450000e-09,245.698287,129.604846,0.150041,0.001585,0.000046,1.080000e-06,7.976625


In [6]:
print(list(df_hok))

['DATE', 'TIME', 'S1[MW]', 'S2[MW]', 'solar[kW/m2]', 'temp[deg C]', 'solar[kW/m2].1', 'temp[deg C].1', 'solar[kW/m2].2', 'temp[deg C].2', 'solar[kW/m2].3', 'temp[deg C].3', 'u10', 'v10', 't2m', 'asn', 'sd', 'ssrdc', 'ssrd', 'tcc', 'tciw', 'tclw', 'tp', 'wind_speed', 'u10.1', 'v10.1', 't2m.1', 'asn.1', 'sd.1', 'ssrdc.1', 'ssrd.1', 'tcc.1', 'tciw.1', 'tclw.1', 'tp.1', 'wind_speed.1', 'u10.2', 'v10.2', 't2m.2', 'asn.2', 'sd.2', 'ssrdc.2', 'ssrd.2', 'tcc.2', 'tciw.2', 'tclw.2', 'tp.2', 'wind_speed.2']


In [7]:
#### df_feats=df_hok[df_hok['ssrdc'] != 0] ## dropping night hours, when clear-sky solar irradiance (SSRDC) forecasts=0, and this can be applied in real forecasts  

#### df_feats=df_hok.set_index('DATE')

############################# Not dropping night hours

df_tar_feats=df_hok.set_index(pd.to_datetime(df_hok['DATE'] + ' ' + df_hok['TIME']))

############## In case of forecast solar power  #######

Targ=df_tar_feats['S1[MW]']+df_tar_feats['S2[MW]']
Targ = Targ.replace(np.nan, 0, regex=True)

df_feats=df_tar_feats[['u10', 'v10', 't2m', 'asn', 'sd', 'ssrdc', 'ssrd', 'tcc', 'tciw', 'tclw', 'tp', 'wind_speed', 
             'u10.1', 'v10.1', 't2m.1', 'asn.1', 'sd.1', 'ssrdc.1', 'ssrd.1', 'tcc.1', 'tciw.1', 'tclw.1', 'tp.1', 'wind_speed.1', 
             'u10.2', 'v10.2', 't2m.2', 'asn.2', 'sd.2', 'ssrdc.2', 'ssrd.2', 'tcc.2', 'tciw.2', 'tclw.2', 'tp.2', 'wind_speed.2']]
############### In case of forecast solar irradiance  #######

# Targ=df_tar_feats['ssrd']  #ssrd is the mean of ssrd data from all ensemble members
# df_feats=df_tar_feats[['u10', 'v10', 't2m', 'asn', 'sd', 'ssrdc', 'tcc', 'tciw', 'tclw', 'tp', 'wind_speed', 
#              'u10.1', 'v10.1', 't2m.1', 'asn.1', 'sd.1', 'ssrdc.1', 'tcc.1', 'tciw.1', 'tclw.1', 'tp.1', 'wind_speed.1', 
#              'u10.2', 'v10.2', 't2m.2', 'asn.2', 'sd.2', 'ssrdc.2', 'tcc.2', 'tciw.2', 'tclw.2', 'tp.2', 'wind_speed.2']]


df_tar_feats.insert(loc=0, column='Targ', value=Targ)

df_feats.head()
df_feats.tail()

print(len(list(df_feats)), len(list(df_tar_feats)))
# Targ.tail()


36 49


In [8]:
len(df_feats)
# len(Targ)
print(len(Targ), max(Targ), Targ.mean())

52603 312.6 37.385689029143066


In [9]:
# ind_date_time= pd.date_range('2019-03-08 00:00','2019-03-09 00:00',freq='120T')
ind_date_time= pd.date_range('2019-03-08 04:00','2019-03-09 03:30',freq='30T')

Hours= ind_date_time.strftime('%H:%M')
# Hours
# for num in range(0,len(Hours),6):
#     print(Hours[num])

In [10]:
df_intrv0=df_tar_feats.between_time('20:00','03:30')
df_intrv1=df_tar_feats.between_time('04:00','04:30')
df_intrv2=df_tar_feats.between_time('05:00','05:30')
df_intrv3=df_tar_feats.between_time('06:00','06:30')
df_intrv4=df_tar_feats.between_time('07:00','07:30')
df_intrv5=df_tar_feats.between_time('08:00','08:30')
df_intrv6=df_tar_feats.between_time('09:00','09:30')
df_intrv7=df_tar_feats.between_time('10:00','10:30')
df_intrv8=df_tar_feats.between_time('11:00','11:30')
df_intrv9=df_tar_feats.between_time('12:00','12:30')
df_intrv10=df_tar_feats.between_time('13:00','13:30')
df_intrv11=df_tar_feats.between_time('14:00','14:30')
df_intrv12=df_tar_feats.between_time('15:00','15:30')
df_intrv13=df_tar_feats.between_time('16:00','16:30')
df_intrv14=df_tar_feats.between_time('17:00','17:30')
df_intrv15=df_tar_feats.between_time('18:00','18:30')
df_intrv16=df_tar_feats.between_time('19:00','19:30')
# df_intrv16.head(9)

In [11]:
#df=dfdfd #Just to stop before run the following cell

In [12]:
from Hokk_Hourly_Forecast_Models_16hrs import model_intrv0, model_intrv1, model_intrv2, model_intrv3, model_intrv4, model_intrv5, model_intrv6, model_intrv7, model_intrv8, model_intrv9, model_intrv10, model_intrv11, model_intrv12, model_intrv13, model_intrv14, model_intrv15, model_intrv16
##### from Hokk_Hourly_Forecast_Models_16hrs_Tuned_By_CV import model_intrv0, model_intrv1, model_intrv2, model_intrv3, model_intrv4, model_intrv5, model_intrv6, model_intrv7, model_intrv8, model_intrv9, model_intrv10, model_intrv11, model_intrv12, model_intrv13, model_intrv14, model_intrv15, model_intrv16

intvs=[df_intrv0, df_intrv1, df_intrv2, df_intrv3, df_intrv4, df_intrv5, df_intrv6, df_intrv7, df_intrv8,
       df_intrv9, df_intrv10, df_intrv11, df_intrv12, df_intrv13, df_intrv14, df_intrv15, df_intrv16]

for num in range(0, len(intvs)):
    df_hr=intvs[num]
#     print(num, len(df_hr), max(df_hr['Targ']), df_hr['Targ'].mean(), df_hr['Targ'].std())
    
    ####Select features of the data 
    Targ=df_hr['Targ']
    df_feats_w_mes=df_hr.drop('Targ', axis=1) #Features including measurements and weather forecasts
    df_feats_no_mes =df_hr.drop(df_tar_feats.columns[range(0,13)], axis=1)  #Features of weather forecasts only

    # df_feats=df_feats_w_mes  ## the measurements lenght is less than the weather since there are not exist in forecast period
    
#     df_feats_no_mes =df_feats_no_mes.drop('asn', axis=1)  #drop albedo snow
#     df_feats_no_mes =df_feats_no_mes.drop('sd', axis=1)  #drop snow depth
#     df_feats_no_mes=df_feats_no_mes.drop(['asn','sd', 'asn.1','sd.1','asn.2','sd.2','u10', 
#                           'v10','asn','sd','u10.1', 'v10.1','u10.2', 'v10.2'], axis=1)   
#     df_feats_no_mes=df_feats_no_mes.drop(['asn','sd', 'asn.1','sd.1','asn.2','sd.2'], axis=1)
#     df_feats_no_mes=df_feats_no_mes.drop(['tciw', 'tclw',  'tciw.1', 'tclw.1', 'tciw.2', 'tclw.2'], axis=1)
#     df_feats_no_mes=df_feats_no_mes.drop(['u10.1', 'v10.1', 't2m.1', 'asn.1', 'sd.1', 'ssrdc.1', 'ssrd.1', 'tcc.1', 'tciw.1', 'tclw.1', 'tp.1', 'wind_speed.1', 
#              'u10.2', 'v10.2', 't2m.2', 'asn.2', 'sd.2', 'ssrdc.2', 'ssrd.2', 'tcc.2', 'tciw.2', 'tclw.2', 'tp.2', 'wind_speed.2'], axis=1)
#########################################


    
    df_feats=df_feats_no_mes   
#     X_train=df_feats.loc['1/1/2017':'12/31/2017']
#     X_test=df_feats.loc['1/1/2016':'12/31/2016']
#     X_train=df_feats.loc['1/1/2016':'12/31/2016']
#     X_test=df_feats.loc['1/1/2017':'12/31/2017']

    X_train=df_feats.loc['1/1/2017':'12/31/2017']
    X_test=df_feats.loc['1/1/2018':'12/31/2018']

#     Y_train=Targ.loc['1/1/2017':'12/31/2017']
#     Y_test=Targ.loc['1/1/2016':'12/31/2016']
#     Y_train=Targ.loc['1/1/2016':'12/31/2016']
#     Y_test=Targ.loc['1/1/2017':'12/31/2017']
    
    Y_train=Targ.loc['1/1/2017':'12/31/2017']
    Y_test=Targ.loc['1/1/2018':'12/31/2018']

    X_train.head()
    # X_train.tail()
    # len(X_train)

    # Y_train.head()
    # len(Y_train)
    Y_train.tail()
    
    if num==0:
        print('Yes20:00-3:30')
        reslts=model_intrv0(Y_train, X_train, Y_test, X_test, Targ)
        st0={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv0=reslts['Yts_pd']    
    elif num==1:
        print('Yes04:00-04:30')
        reslts=model_intrv1(Y_train, X_train, Y_test, X_test, Targ)
        st1={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv1=reslts['Yts_pd'] 
    elif num==2:
        print('Yes05:00-05:30')
        reslts=model_intrv2(Y_train, X_train, Y_test, X_test, Targ) 
        st2={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv2=reslts['Yts_pd']  
    elif num==3:
        print('Yes06:00-06:30')
        reslts=model_intrv3(Y_train, X_train, Y_test, X_test, Targ)
        st3={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv3=reslts['Yts_pd'] 
    elif num==4:
        print('Yes07:00-07:30')
        reslts=model_intrv3(Y_train, X_train, Y_test, X_test, Targ)
        st4={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv4=reslts['Yts_pd']     
    elif num==5:
        print('Yes08:00-08:30')
        reslts=model_intrv2(Y_train, X_train, Y_test, X_test, Targ) 
        st5={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv5=reslts['Yts_pd']  
    elif num==6:
        print('Yes09:00-09:30')
        reslts=model_intrv3(Y_train, X_train, Y_test, X_test, Targ)
        st6={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv6=reslts['Yts_pd'] 
    elif num==7:
        print('Yes10:00-10:30')
        reslts=model_intrv3(Y_train, X_train, Y_test, X_test, Targ)
        st7={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv7=reslts['Yts_pd']     
    elif num==8:
        print('Yes11:00-11:30')
        reslts=model_intrv3(Y_train, X_train, Y_test, X_test, Targ)
        st8={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv8=reslts['Yts_pd']   
    elif num==9:
        print('Yes12:00-12:30')
        reslts=model_intrv1(Y_train, X_train, Y_test, X_test, Targ)
        st9={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv9=reslts['Yts_pd'] 
    elif num==10:
        print('Yes13:00-13:30')
        reslts=model_intrv2(Y_train, X_train, Y_test, X_test, Targ) 
        st10={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv10=reslts['Yts_pd']  
    elif num==11:
        print('Yes14:00-14:30')
        reslts=model_intrv3(Y_train, X_train, Y_test, X_test, Targ)
        st11={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv11=reslts['Yts_pd'] 
    elif num==12:
        print('Yes15:00-15:30')
        reslts=model_intrv3(Y_train, X_train, Y_test, X_test, Targ)
        st12={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv12=reslts['Yts_pd']     
    elif num==13:
        print('Yes16:00-16:30')
        reslts=model_intrv2(Y_train, X_train, Y_test, X_test, Targ) 
        st13={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv13=reslts['Yts_pd']  
    elif num==14:
        print('Yes17:00-17:30')
        reslts=model_intrv3(Y_train, X_train, Y_test, X_test, Targ)
        st14={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv14=reslts['Yts_pd'] 
    elif num==15:
        print('Yes18:00-18:30')
        reslts=model_intrv3(Y_train, X_train, Y_test, X_test, Targ)
        st15={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv15=reslts['Yts_pd']     
    elif num==16:
        print('Yes19:00-19:30')
        reslts=model_intrv3(Y_train, X_train, Y_test, X_test, Targ)
        st16={'mse': reslts['mse'], 'nRMSE':reslts['nRMSE']}
        Ytspd_inv16=reslts['Yts_pd'] 
    else:
        print('No')

Yes20:00-3:30


C:\Users\Mhdella\Google Drive\Jupyter Folder\PV_Hokkaido_forecast_contest\Forecast_codes\Hokk_Hourly_Forecast_Models_16hrs.py:17: RuntimeWarning: invalid value encountered in double_scalars
  nRMSE=np.sqrt(mse)/Targ.mean()


0.0 nan
Yes04:00-04:30
1.7802711274400318 4.402702817959011
Yes05:00-05:30
134.45094979654226 2.757341102375605
Yes06:00-06:30
1532.0005393985264 2.0532541703723477
Yes07:00-07:30
6982.14291455394 1.7499680831302733
Yes08:00-08:30
17959.234628035196 1.6209391444875878
Yes09:00-09:30
29904.718991618083 1.5814043250117071
Yes10:00-10:30
36902.86892223369 1.5692354584969141
Yes11:00-11:30
38687.25983337313 1.5687603516395012
Yes12:00-12:30
37236.08377072271 1.5933452149269858
Yes13:00-13:30
30098.21731753094 1.5925549904476197
Yes14:00-14:30
18098.748984804686 1.6237264827409923
Yes15:00-15:30
7585.354811753149 1.8100857195905584
Yes16:00-16:30
1804.3576510444302 2.125088710574093
Yes17:00-17:30
154.35318710807272 2.62038907863443
Yes18:00-18:30
2.754819753999844 4.048297332989362
Yes19:00-19:30
8.904358254291367e-06 65.40965557098389


In [22]:
sYs=sum([len(Ytspd_inv0), len(Ytspd_inv1), len(Ytspd_inv2), len(Ytspd_inv3), len(Ytspd_inv4),
        len(Ytspd_inv5), len(Ytspd_inv6), len(Ytspd_inv7), len(Ytspd_inv8),
        len(Ytspd_inv9), len(Ytspd_inv10), len(Ytspd_inv11), len(Ytspd_inv12),
        len(Ytspd_inv13), len(Ytspd_inv14), len(Ytspd_inv15), len(Ytspd_inv16)])

print(sYs, len(df_tar_feats['Targ'].loc['1/1/2018':'12/31/2018']))

17520 17520


In [23]:
Ytspd_inv0[Ytspd_inv0>0]=0
Ytspd_inv0[Ytspd_inv0<0]=0

print(len(Ytspd_inv8))
# Ytspd_inv0.head()

730


In [24]:
df_ytgfst= pd.concat([Ytspd_inv0, Ytspd_inv1, Ytspd_inv2, Ytspd_inv3, Ytspd_inv4, Ytspd_inv5, Ytspd_inv6, Ytspd_inv7, Ytspd_inv8,
                     Ytspd_inv9, Ytspd_inv10, Ytspd_inv11, Ytspd_inv12, Ytspd_inv13, Ytspd_inv14, Ytspd_inv15, Ytspd_inv16])
df_ytgfst.sort_index(inplace=True)
df_ytgfst.head()
df_ytgfst.tail()

,Yts,Ypd
2018-12-31 21:30:00,0.0,0.0
2018-12-31 22:00:00,0.0,0.0
2018-12-31 22:30:00,0.0,0.0
2018-12-31 23:00:00,0.0,0.0
2018-12-31 23:30:00,0.0,0.0


In [30]:
df_ytgfst['Yts'].isna().sum()  #Check if there was enteries as Nan in the target and predictions
df_ytgfst['Ypd'].isna().sum()
df_tar_feats['Targ'].mean() 
df_ytgfst['Yts'].mean()

0.0

In [31]:
MSE=mean_squared_error(df_ytgfst['Yts'], df_ytgfst['Ypd'])  
# NRMSE=np.sqrt(MSE)/df_ytgfst['Yts'].mean()  #mean of target solar power during 2017 only
NRMSE=np.sqrt(MSE)/df_tar_feats['Targ'].mean()  #mean of target solar power durning 2016 and 2017

print(MSE, NRMSE)

9461.846983406622 2.6018518910336867


In [32]:
##### Here RMSE is normalized to NRMSE by division of the mean of solar power in the forecasted year
MSE=mean_squared_error(df_ytgfst['Yts'], df_ytgfst['Ypd'])  
NRMSE=np.sqrt(MSE)/df_ytgfst['Yts'].mean() #mean of target solar power durning 2016 and 2017

print(MSE, NRMSE)

9461.846983406622 inf


C:\Users\Mhdella\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


Run XGBoost models of 16 intervals to forecast solar power entire 2017, leads to:

3.552713678800501e-15 inf
Yes04:00-04:30
0.475519379605122 2.275413906134348
Yes05:00-05:30
23.08378863532188 1.142515503418456
Yes06:00-06:30
181.8652749046895 0.7074376122917255
Yes07:00-07:30
647.4780610224576 0.5329031527598782
Yes08:00-08:30
1575.4455398693974 0.4800920292549294
Yes09:00-09:30
2330.0799227751886 0.44142656785601114
Yes10:00-10:30
2487.094226788558 0.40738418685260663
Yes11:00-11:30
2448.7018572767597 0.3946758697832598
Yes12:00-12:30
2282.52364338894 0.3944895684037349
Yes13:00-13:30
1976.5856534554887 0.40811435409543745
Yes14:00-14:30
1351.062692232559 0.44363555627627355
Yes15:00-15:30
651.7656363669407 0.5305873866634264
Yes16:00-16:30
146.10163921422046 0.604704632561749
Yes17:00-17:30
21.76416924077891 0.9839635874614143
Yes18:00-18:30
1.2307041749359056 2.705843317430681
Yes19:00-19:30
1.5638608410656872e-05 86.68413151347968

For 2017 foreccasts: print(MSE, NRMSE)=671.8857643485188 0.4524476931829704 (<0.4605 in four intervals case)
When normalizing nRMSE by mean of all power output: print(MSE, NRMSE)=671.8857643485188, 0.6933337329108594(<0.705803, imporve 1.76%)
The most challenging intervals, are noon and afternoon hours 10 to 14. with higher MSEs (nRMSE is not valid here to indicate which interval is most challenging because it is nomralized by division of mean of power, so in mornings and evenings the mean become lower and then the nRMSE)

After tuning XGBoost's parameters, the forecasts improved slightly (becuase the previous parameters are from the 8 intervals, which are already from the CV of XGBoost)

print(MSE, NRMSE)=682.8464839906529 0.45612323653247416 (<0.4605) 

NOTE: The NRMSE of XGB models that are tuned by CV is slightly higher than the XGBs that have the parameters of 8-interval-tunning parameters (0.45612323653247416~>0.4524476931829704),although it is a small difference=0.8%.'
Also for 2016 using XGBs with new 16 interval  tunned-paramters leads to slightly higher nRMSE: print(MSE, NRMSE)=755.9593927930301,  0.5011357219226292 comapared to the case of using XGBs with  8-intervals-tuned paramters: print(MSE, NRMSE)=735.9314955977709,  0.4944527792392538

Therefore, the models of previous CV 8-interval-tuned parameter are which adopted here, while the new models parameter are saved in another file, called as: (Hokk_Hourly_Forecast_Models_16hrs_Tuned_By_CV.py)

Improvement over the 8 intervals case: (0.46-0.45)/0.46*100=2.17%

Improvement over one model for all hours case (no separate models for every interval):(1.2399-0.693333)/1.2399*100
=44.08%

If RMSE is normalized to NRMSE by division of the mean of solar power in the forecasted year
NRMSE=np.sqrt(MSE)/df_ytgfst['Yts'].mean()

For 2016: print(MSE, NRMSE)=735.9314955977709,  0.4944527792392538

For 2017: print(MSE, NRMSE)=6671.8857643485188,  0.4524476931829704

While for 2108, the actual solar power output is not known, but in the tentative ranking of results in contest website, for 2018 forecast is first ranked is NRMSE=0.498  https://cuusoo.com/predictions/123

# Note:
Do not forget to uncomment the option of 2018 (X_train, X_test and Y_train, Y_test), when submitting the forecasts of solar power in 2018 (not 2017 or 2016) for Hokkaido island in Japan to the contest secretariat.
    # Y_train=Targ.loc['1/1/2017':'12/31/2017']
    # Y_test=Targ.loc['1/1/2018':'12/31/2018']
    
Wihtout albedo snow (asn) and snow depth (sd) variables: foreccasts in 2017 has NRMSE=0.4509 < 0.4524 (imporv=0.35%)

Without asn, sd, from all ensembles (mean, max min), NRMSE=0.4561186593208066(>0.4524)

Without asn, sd, u10, v10, from all ensembles (mean, max min), NRMSE=0.45997067690183424(>0.4524)

Without  total cloud ice and water content (tciw and tclw), NRMSE=0.46527322388508574(>0.4524)

Without  the maimum and minimum values of variables of ensemble members, just usign the mean values of 12 variables of emsemble members, NRMSE=0.4559069170772983(>0.4524)

In Hokk_Sol_Pw_Forecasts_16Hours_ssrd_lead_lag.ipyn ssrd lead+1,lead+2 and lag-1, lag-1 are included by they are just slightly imporved the results:
for 2017: print(MSE, NRMSE)=668.646926770953 0.45135585961527724(<0.4524), imporve=0.24%
for 2016(with lead and lags of ssrd)print(MSE, NRMSE=735.4564339689094 0.4942931628455824(~=0.4944527792392538)


0.693333*df_tar_feats['Targ'].mean()/max(df_tar_feats['Targ'])=0.0829%

In [19]:
print(len(Ytspd_inv0), len(Ytspd_inv1), len(Ytspd_inv2), len(Ytspd_inv3), len(Ytspd_inv4))

5840 730 730 730 730


In [20]:
0.693333*df_tar_feats['Targ'].mean()/max(df_tar_feats['Targ']) 

0.08291980784274743

In [21]:
0.452447*df_ytgfst['Yts'].mean()/max(df_ytgfst['Yts']) 

ZeroDivisionError: float division by zero

In [ ]:
(0.70580-0.693333)/0.70580*100

In [ ]:
df_ytgfst['Yts'].mean()

In [ ]:
(1.2399-0.693333)/1.2399*100

In [ ]:
(0.46-0.45)/0.46

In [ ]:
(0.45612323653247416-0.4524476931829704)/0.45612323653247416


In [ ]:
(0.4524476931829704-0.450862420876339)/0.4524476931829704*100

In [ ]:
print(len(list(X_test)))
X_test.head()